# IBM Watson Library

Full details and documentation on the IBM Watson library, check it ou here: https://pypi.org/project/ibm-watson/

In [ ]:
#  pip install --upgrade ibm-watson
#  pip install python-dotenv

In [ ]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

import json

In [ ]:
# Store environment variables in an external .dotenv file
# This means we don't have to share it when we share the notebook (like this)
# You can add your own .env file adjacent to this .ipynb notebook
# OR just replace the APIKEY and SERVICE_URL in the notebook with the ones you copy from IBM Cloud

from os import path, getenv
from dotenv import load_dotenv

# Visual Recognition

### Load environment variables

In [ ]:
# We're setting the path of our environment variable (dotenv_path)
# Then loading from that path

# path.abspath('') returns the name of the folder this notebook runs in
# path.join() constructs the filepath

dotenv_path = path.join(path.abspath(''), '.env')
load_dotenv(dotenv_path)

### Authenticate and Set Service URL

In [ ]:
# We use getenv() to get the string that our environment variable references
authenticator = IAMAuthenticator(getenv('VIS_APIKEY'))
authenticator

In [ ]:
vis_recog = VisualRecognitionV3(
    version = '2018-03-19',
    authenticator = authenticator
)
vis_recog

In [ ]:
vis_recog.set_service_url(getenv('VIS_SERVICEURL'))

### Set image

In [ ]:
img_url = 'https://thefunnybeaver.com/wp-content/uploads/2017/07/angry-otter.jpg'

### Call service

In [ ]:
classes_result = vis_recog.classify(url = img_url).get_result()

In [ ]:
classes_result

In [ ]:
print(json.dumps(classes_result, indent=2))

### Quick function

In [ ]:
def nice_classify(url):
    classes_result = vis_recog.classify(url = url).get_result()
    print(json.dumps(classes_result, indent=2))

## Built-In Classifiers

https://cloud.ibm.com/apidocs/visual-recognition/visual-recognition-v3?code=python#classify <br>
Check for explicit?!?!

In [ ]:
nice_classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.ytimg.com%2Fvi%2FqRPC6vOrIzE%2Fmaxresdefault.jpg&f=1&nofb=1')

In [ ]:
def classify(url, classifier = ['default']):
    classes_result = vis_recog.classify(
        url = url,
        classifier_ids = classifier
    ).get_result()
    return classes_result

In [ ]:
classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.ytimg.com%2Fvi%2FqRPC6vOrIzE%2Fmaxresdefault.jpg&f=1&nofb=1', classifier = ['default', 'food', 'explicit'])

## Custom Classifier

Full details here: https://cloud.ibm.com/apidocs/visual-recognition/visual-recognition-v3?code=python#createclassifier

### Create

In [ ]:
# If there are too many custom classifiers already, you may need to delete some (see below)

with open('./Training Images/Mantas.zip', 'rb') as mantas, open('./Training Images/Stingrays.zip', 'rb') as stingrays, open('./Training Images/Otters.zip', 'rb') as otters:
    model = vis_recog.create_classifier(
        name = 'Rays',
        positive_examples = {'manta_ray': mantas, 'stingray': stingrays},
        negative_examples = otters
    ).get_result()

In [ ]:
print(json.dumps(model, indent=2))

### List Classifiers

In [ ]:
classifiers = vis_recog.list_classifiers(verbose=True).get_result()
print(json.dumps(classifiers, indent=2))

In [ ]:
# This shows all the custom classifiers in the list
for classifier in classifiers['classifiers']:
    print(classifier['classifier_id'])

### Delete Classifier

In [ ]:
vis_recog.delete_classifier('Rays_1361634687')

There are also APIs that support:
- retrieving a list of available classifiers
- retrieving classifier details
- updating classifieres
- deleting classifiers

### Use

In [ ]:
ray_classified = classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmedia.buzzle.com%2Fmedia%2Fimages-en%2Fgallery%2Ffish%2F1200-55844925-stingray-grand-cayman-island.jpg&f=1&nofb=1', ['default', 'Batoidea_848670369'])

In [ ]:
ray_classified

# Text to Speech

In [ ]:
from ibm_watson import TextToSpeechV1

In [ ]:
TTSauthenticator = IAMAuthenticator(getenv('TTS_APIKEY'))
text_to_speech = TextToSpeechV1(
    authenticator = TTSauthenticator
)

text_to_speech.set_service_url(getenv('TTS_SERVICEURL'))

## List Voices

In [ ]:
voices = text_to_speech.list_voices().get_result()

In [ ]:
print(json.dumps(voices, indent=2))

## Synthesise Voice

In [ ]:
synthesis = text_to_speech.synthesize(
    text = 'Buenos dias amigos. Como estas?',
    voice = 'es-ES_EnriqueV3Voice',
    accept='audio/wav'
).get_result()

In [ ]:
synthesis

In [74]:
with open('./TTS/TextToSpeech Output.wav', 'wb') as audio_test:
    audio_test.write(synthesis.content)

# Speech to Text

In [ ]:
from ibm_watson import SpeechToTextV1

In [ ]:
authenticator = IAMAuthenticator(getenv('STT_APIKEY'))
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url(getenv('STT_SERVICEURL'))

### List Models and Get Details

In [ ]:
speech_models = speech_to_text.list_models().get_result()
print(json.dumps(speech_models, indent=2))

In [ ]:
speech_model = speech_to_text.get_model('en-US_BroadbandModel').get_result()
print(json.dumps(speech_model, indent=2))

### Recognise Speech

In [77]:
# Use the Trump person-woman-man-camera-tv
audio_location = path.join(path.abspath(''), 'STT/Person Woman Man Camera TV.mp3')
audio_location

'/Users/chungleu@uk.ibm.com/Documents/IBM/C&A/Cognitive Technical Community/Data Science 101/107 - Watson Services/STT/Person Woman Man Camera TV.mp3'

In [78]:
with open(audio_location, 'rb') as audio_file:
    transcript = speech_to_text.recognize(
            audio=audio_file,
            content_type='audio/mp3',
            model='en-US_BroadbandModel'
        ).get_result()

### Transform Transcript to JSON and Write to File

In [83]:
json_transcript = json.dumps(transcript, indent=2)

In [85]:
with open('./STT/TrumpTranscript.json', 'w') as outputfile:
    outputfile.write(json_transcript)